---
<p align="center">
  <img src="https://www.portafolio.co/files/article_new_multimedia/uploads/2023/10/12/652828e536642.jpeg" style="width:100%; height:120px; object-fit:cover;"/>
</p>

<h1 align="center" style="color:#FFD700;"> <span style="color:#FFD700;">Optimización del Transporte en Nueva York</span> </h1>

<p align="center">
  <b>Green Taxi | Machine Learning & Data Science | CRISP-DM</b><br>
  <span style="font-size:1.1em;">Análisis y predicción de tarifas y duración de viajes usando datos reales de taxis verdes de NYC.</span>
</p>

---


##  Contexto y problema
La Comisión de Taxis y Limusinas de Nueva York (TLC) dispone de registros masivos de viajes de taxis verdes (Green Taxi). Por ejemplo, la distancia promedio de viaje es de aproximadamente 2.5 millas, la tarifa media ronda los $12 y existen recargos variables según la zona y el horario.

El reto principal es que las tarifas y duraciones varían significativamente según la hora, la distancia y la zona. Los pasajeros buscan información precisa para planificar sus traslados y la TLC necesita optimizar rutas y recursos.

Este análisis no solo busca resolver problemas inmediatos como la estimación de tarifas y tiempos, sino también proporcionar una base sólida para la planificación estratégica a largo plazo. Al comprender los patrones de movilidad, se pueden identificar áreas de alta demanda, optimizar la asignación de recursos y mejorar la experiencia del usuario. Además, este enfoque permite detectar anomalías y garantizar que los datos sean utilizados de manera efectiva para tomar decisiones informadas.

##  Objetivos de negocio
- Predecir la tarifa total (`fare_amount` + recargos) y la duración estimada del viaje (`trip_duration` en minutos) con precisión suficiente para informar a pasajeros y optimizar la asignación de vehículos.
- Mejorar la transparencia y confianza del pasajero, ofreciendo estimaciones basadas en datos reales.
- Ayudar a la TLC y operadores a anticipar demanda y reducir tiempos de espera.
- Detectar anomalías (ej: viajes con `fare_amount` negativo o `trip_distance` > 100 millas).

##  Stakeholders
- TLC (regulador): políticas y planificación.
- Operadores (empresas de dispatch): asignación y flotas.
- Conductores: optimización de rutas y tiempos activos.
- Pasajeros: estimaciones de coste/tiempo al reservar.
- Equipo de datos/ML: entrega de modelos y monitoreo.

##  Criterios de éxito / métricas
- MAE y RMSE para duración (`trip_duration`) y tarifa (`fare_amount` + recargos).
- Ejemplo orientativo: MAE de duración < 5 minutos; MAE de tarifa < $2.
- % de predicciones dentro de ±10% del valor real; cobertura por zona/hora; latencia de inferencia < 500 ms.

##  Preguntas clave
- ¿Qué variables explican mejor la duración y la tarifa? (ej: `trip_distance`, `pickup_hour`, `RatecodeID`, `payment_type`)
- ¿Cómo afectan los recargos (`congestion_surcharge`, `cbd_congestion_fee`) la tarifa total?
- ¿Existen patrones horarios o estacionales que condicionen la precisión del modelo?
- ¿Qué zonas (`PULocationID`, `DOLocationID`) presentan mayor demanda o tarifas atípicas?

##  Fuentes de datos y variables relevantes
- Dataset principal: [Green Taxi Trip Records]

    https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-06.parquet

- Variables clave: `lpep_pickup_datetime`, `lpep_dropoff_datetime`, `trip_distance`, `fare_amount`, `tip_amount`, `tolls_amount`, `congestion_surcharge`, `cbd_congestion_fee`, `passenger_count`, `payment_type`, `RatecodeID`, `PULocationID`, `DOLocationID`.
- Ejemplo: la mediana de `fare_amount` es $10, la mayoría de los viajes tienen `trip_distance` entre 1 y 5 millas, y los recargos pueden sumar hasta $5 adicionales.

##  Supuestos y limitaciones
- Supuesto: timestamps en zona horaria consistente; variables numéricas sin valores negativos ni nulos tras limpieza.
- Limitación: registros con `trip_distance` <= 0, `fare_amount` <= 0 o `trip_duration` <= 0 serán eliminados.
- Eventos extraordinarios (clima extremo, huelgas) pueden afectar la validez de modelos entrenados con datos históricos.

##  Riesgos
- Cambios en el esquema de datos (nuevos cargos a partir de 2025).
- Sesgos geográficos o por horario que reduzcan la capacidad de generalización.
- Calidad de datos insuficiente para alcanzar las métricas objetivo.

In [3]:
import os
import pandas as pd

# URL del dataset
url = "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-06.parquet"

# Cargar el archivo parquet desde la URL
data = pd.read_parquet(url, engine="pyarrow")

In [ ]:
data.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,cbd_congestion_fee
0,2,2025-06-01 00:33:43,2025-06-01 01:04:33,N,2.0,74,132,1.0,19.60,70.0,...,0.5,19.61,6.94,NaN,1.0,98.05,1.0,1.0,0.0,0.0
1,2,2025-06-01 00:07:45,2025-06-01 00:14:52,N,1.0,75,74,2.0,1.37,9.3,...,0.5,0.00,0.00,NaN,1.0,11.80,2.0,1.0,0.0,0.0
2,2,2025-06-01 00:24:07,2025-06-01 00:48:24,N,1.0,83,83,1.0,4.11,25.4,...,0.5,0.00,0.00,NaN,1.0,27.90,2.0,1.0,0.0,0.0
3,2,2025-06-01 00:00:14,2025-06-01 00:08:29,N,1.0,97,49,1.0,1.29,9.3,...,0.5,2.36,0.00,NaN,1.0,14.16,1.0,1.0,0.0,0.0
4,2,2025-06-01 00:31:15,2025-06-01 00:43:35,N,1.0,66,25,1.0,1.97,13.5,...,0.5,0.00,0.00,NaN,1.0,16.00,1.0,1.0,0.0,0.0
